In [1]:

import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from skimage.metrics import structural_similarity as ssim
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image

# Constants
IMAGE_SIZE = (128, 128)
DATASET_PATH = "D:/CNN Project/mvtec_anomaly_detection"
PROCESSED_PATH = "processed_data"
os.makedirs(PROCESSED_PATH, exist_ok=True)

print("Libraries Imported Successfully!")


Libraries Imported Successfully!


In [2]:

def preprocess_image(image_path):
    """Load and preprocess image for model input."""
    if not os.path.isfile(image_path):
        print(f"Skipping non-file: {image_path}")
        return None

    image = cv2.imread(image_path, cv2.IMREAD_COLOR)  
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  
    image = cv2.resize(image, IMAGE_SIZE)  
    image = image.astype(np.float32) / 255.0  
    return np.expand_dims(image, axis=0)  # Add batch dimension

print("Preprocessing function ready!")


Preprocessing function ready!


In [3]:
def load_good_images():
    """Load only 'good' images from the dataset."""
    data, labels = [], []
    category_path = os.path.join(DATASET_PATH, "bottle", "train", "good")

    if not os.path.exists(category_path) or not os.path.isdir(category_path):
        print("Path not found:", category_path)
        return np.array([]), np.array([])

    for img_name in os.listdir(category_path):
        img_path = os.path.join(category_path, img_name)
        if is_image_file(img_name):
            image = preprocess_image(img_path)
            if image is not None:
                data.append(image)
                labels.append(0)  # Normal images labeled as 0

    return np.array(data), np.array(labels)

# Load the data
data, labels = load_good_images()
print(f"Total Images Loaded: {len(data)}")

# Train-test split
train_images, val_images, train_labels, val_labels = train_test_split(
    data, labels, test_size=0.2, random_state=42
)

# Save preprocessed data
np.save(os.path.join(PROCESSED_PATH, "train_images.npy"), train_images)
np.save(os.path.join(PROCESSED_PATH, "val_images.npy"), val_images)

print("Preprocessing complete. Data saved successfully.")

NameError: name 'is_image_file' is not defined

In [14]:
train_images.shape, val_images.shape

((167, 1, 128, 128, 3), (42, 1, 128, 128, 3))

In [15]:

def build_autoencoder():
    """Create a Convolutional Autoencoder Model."""
    input_img = keras.Input(shape=(128, 128, 3))

    # Encoder
    x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input_img)
    x = layers.MaxPooling2D((2, 2), padding="same")(x)
    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D((2, 2), padding="same")(x)

    # Bottleneck
    x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)

    # Decoder
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
    x = layers.Conv2D(3, (3, 3), activation="sigmoid", padding="same")(x)

    model = keras.Model(input_img, x)
    model.compile(optimizer="adam", loss="mse")
    return model

# Build model
autoencoder = build_autoencoder()
autoencoder.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_2 (UpSampling2D)  │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 64, 64, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_3 (UpSampling2D)  │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 128, 128, 32)   │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 128, 128, 3)    │           867 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 186,371 (728.01 KB)

 Trainable params: 186,371 (728.01 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:

# Load data
train_images = np.load(os.path.join(PROCESSED_PATH, "train_images.npy"))
val_images = np.load(os.path.join(PROCESSED_PATH, "val_images.npy"))

# Train the model
history = autoencoder.fit(train_images, train_images, 
                          epochs=50, batch_size=16, 
                          validation_data=(val_images, val_images))

# Save the model
#autoencoder.save("autoencoder_model.h5")
print("Model training complete and saved!")


Epoch 1/50


ValueError: Creating variables on a non-first call to a function decorated with tf.function.

In [ ]:

def compute_loss(original, reconstructed):
    """Compute SSIM loss for anomaly detection."""
    original = np.squeeze(original)  
    reconstructed = np.squeeze(reconstructed)  
    return 1 - ssim(original, reconstructed, data_range=1.0, channel_axis=-1)

# Get losses for good images
good_losses = [compute_loss(img, autoencoder.predict(np.expand_dims(img, axis=0))) for img in val_images]

# Set threshold using percentile method
threshold = np.percentile(good_losses, 95)  # 95th percentile

print(f"Computed Anomaly Threshold: {threshold}")


In [ ]:

def predict_anomaly(image_path):
    """Predict if an image is defective or good."""
    image = preprocess_image(image_path)
    if image is None:
        return "Invalid image!"

    reconstructed = autoencoder.predict(image)
    loss = compute_loss(image, reconstructed)

    if loss > threshold:
        return f"🔴 Defective Image Detected! (Loss: {loss:.4f})"
    else:
        return f"🟢 Good Image (Loss: {loss:.4f})"

# Example usage
test_image_path = "D:/CNN Project/mvtec_anomaly_detection/bottle/test/defective/000.png"
print(predict_anomaly(test_image_path))


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt

# 1. Load and preprocess data
(x_train, _), (x_test, _) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 2. Define the CNN autoencoder architecture
def create_cnn_autoencoder(input_shape=(32, 32, 3), latent_dim=64):
    input_img = layers.Input(shape=input_shape)

    # Encoder
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(latent_dim, (3, 3), activation='relu', padding='same')(x)
    encoded = layers.MaxPooling2D((2, 2), padding='same')(x) # latent space

    # Decoder
    x = layers.Conv2D(latent_dim, (3, 3), activation='relu', padding='same')(encoded)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x) #sigmoid for 0-1 range.

    autoencoder = models.Model(input_img, decoded)
    return autoencoder

# 3. Create and compile the model
autoencoder = create_cnn_autoencoder()
autoencoder.compile(optimizer='adam', loss='mse')

# 4. Train the model
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test))

# 5. Evaluate and visualize results
decoded_imgs = autoencoder.predict(x_test)

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()



170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 77s 0us/step
Epoch 1/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 224s 525ms/step - loss: 0.0264 - val_loss: 0.0110
Epoch 2/50
 16/391 ━━━━━━━━━━━━━━━━━━━━ 1:54 306ms/step - loss: 0.0114

In [ ]:
#Example of changing latent dimension, and adding batch normalization
def create_cnn_autoencoder_improved(input_shape=(32, 32, 3), latent_dim=32):
    input_img = layers.Input(shape=input_shape)

    # Encoder
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(latent_dim, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = layers.Conv2D(latent_dim, (3, 3), activation='relu', padding='same')(encoded)
    x = layers.BatchNormalization()(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.UpSampling2D((2, 2))(x)
    decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = models.Model(input_img, decoded)
    return autoencoder